In [2]:
from scipy.signal import hilbert
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

### FUNCTIONS:

In [ ]:
def find(name, path='/data/i3home/ssued/RNOGCnn'):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
print(f'{find('event_dict.pkl')}')

In [4]:
def shift_relative_time(event, plot=False):
    """
    Shift all channels in an event by the value of the first trigger signal time.

    NOTE: Might be too expensive, as you have to search throughout all of the time arrays to find the minimum time value.
    Perhaps its better to not search? Looking at the generated voltage vs. time arrays, all channel seem to have the same starting time?

    Parameters:
    event (np.array): Event containing channels with voltage vs. time arrays.
    plot (bool): If True, generates before and after subplots (for debugging purposes).
    """
    
    # Plot before the shift, if requested
    if plot:
        _, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].plot(event[0][0], event[0][1])
        ax[0].set_title('Before Shift')
        ax[0].set_xlabel('Time')
        ax[0].set_ylabel('Voltage')

    # Find the minimum time across all channels
    min_time = min(np.min(channel[0]) for channel in event)

    # Subtract min_time from the time values of each channel
    for channel in event:
        channel[0] -= min_time

    # Plot after the shift, if requested
    if plot:
        ax[1].plot(event[0][0], event[0][1])
        ax[1].set_title('After Shift')
        ax[1].set_xlabel('Time (ns)')
        ax[1].set_ylabel('Voltage (V)')
        plt.tight_layout()
        plt.savefig('Before_and_after_shift.png')

In [5]:
def bin_v(channel, nbins, hilbert = False, method='max', plot=False):
    """
    Bin voltage vs. time data. The bin values are calculated according to different methods.

    A big issue is that if the different channels effectively HAVE different time domains, then the different channels will contain
    different bin sizes. We could make all bins the same size, but then graphs will have more or less bins, or we could keep it like this.
    Parameters:
    channel (np.array): Channel array containing two numpy arrays of time (pos [0]) and voltage (pos [1]).
    method (str): Modifies bin values according to the method. 
                  'max' - Bins Hilbert envelope according to the local maximum of their respective time bins.
                  'avg' - Bins Hilbert envelope according to the average of their respective time bins.
    plot (bool): If True, plots the original Hilbert envelope and the binned data.

    Returns:
    np.array: Binned time array
    np.array: Binned Hilbert envelope array
    float: Time of a bin.
    """
    
    time_arr = channel[0]
    voltage_arr = channel[1]

    if hilbert is True:
        # Obtain Hilbert envelope
        v_arr = np.abs(hilbert(voltage_arr))
    else:
        v_arr = voltage_arr

    bin_width = len(voltage_arr) // nbins
    bin_dt = time_arr[bin_width]-time_arr[0]
    binned_time = time_arr[:nbins * bin_width:bin_width]

    if method == 'max':
        binned = np.array([np.max(v_arr[i:i+bin_width]) for i in range(0, nbins * bin_width, bin_width)])
    elif method == 'avg':
        binned = np.array([np.mean(v_arr[i:i+bin_width]) for i in range(0, nbins * bin_width, bin_width)])
    else:
        raise ValueError(f"Invalid method '{method}'. Use 'max' or 'avg'.")

    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(time_arr, v_arr, label='Original Hilbert Envelope', alpha=0.7)
        plt.plot(binned_time, binned, 'o-', label=f'Binned Hilbert Envelope ({method})', color='red')
        plt.xlabel(f'Time (ns), one bin = {bin_dt} ns')
        plt.ylabel('Hilbert Envelope (V)')
        plt.title('Original vs Binned Hilbert Envelope')
        plt.legend()
        plt.grid(True)
        plt.savefig('Original vs Binned Hilbert Envelope.png')

    return binned_time, binned, bin_dt

In [6]:
def calculate_noise(voltage_arr, plot=False):
    """
    Calculate the RMS noise of a voltage array and optionally plot the voltage data.

    The noise is calculated by:
    1. Identifying the peak signal (maximum value) in the array.
    2. Defining a range around the peak, covering 25% of the array's length to the left and right of the peak.
    3. Extracting the noise subarrays from the portions of the array outside this range.
    4. Calculating the RMS noise from the concatenated noise subarrays.

    Parameters:
    voltage_arr (np.array): Array of voltage values.
    plot (bool): If True, plots the voltage data with noise range markers (For debugging purposes).

    Returns:
    float: The RMS noise value.
    """
    
    peak_idx = np.argmax(voltage_arr)
    quarter_len = len(voltage_arr) // 4

    # Define the range around the peak signal
    left_idx = max(0, peak_idx - quarter_len)
    right_idx = min(len(voltage_arr), peak_idx + quarter_len)

    # Extract noise subarrays and concatenate them
    noise_subarray = np.concatenate((voltage_arr[:left_idx], voltage_arr[right_idx:]))

    # Calculate RMS noise
    rms_noise = np.sqrt(np.mean(noise_subarray ** 2))

    # For testing purposes:
    if plot:
        # Plotting the original voltage data
        plt.figure(figsize=(10, 6))
        plt.plot(voltage_arr, label='Voltage Data', color='blue')
        
        # Mark the start and end indices of the noise range
        plt.axvline(x=left_idx, color='red', linestyle='--', label='Left Noise Index')
        plt.axvline(x=right_idx, color='green', linestyle='--', label='Right Noise Index')
        
        # Highlighting the noise areas on the plot
        plt.fill_between(range(left_idx), voltage_arr[:left_idx], color='red', alpha=0.3)
        plt.fill_between(range(right_idx, len(voltage_arr)), voltage_arr[right_idx:], color='green', alpha=0.3)
        
        plt.xlabel('Index')
        plt.ylabel('Voltage (V)')
        plt.title('Voltage Data with Noise Range')
        plt.legend()
        plt.grid(True)
        plt.savefig('Voltage Data with Noise Range.png')

    return rms_noise

In [7]:
def bin_matrix(event,bins = 25, plotting = False, hilb = False):
    row1 = bin_v(event['data'][0],bins, hilbert = hilb)[1]
    row2 = bin_v(event['data'][1],bins, hilbert = hilb)[1]
    row3 = bin_v(event['data'][2],bins, hilbert = hilb)[1]
    row4 = bin_v(event['data'][3],bins, hilbert = hilb)[1]
    bintime = row1[3]
    bintime_ns = bintime*(10**6)
    matrix = np.array([row1,row2,row3,row4])

    binned_dict = {'data' : matrix, 'bintime' : bintime_ns}

    if plotting:
        plt.figure(figsize=(10, 6))
        # Creating the heatmap
        plt.imshow(matrix, cmap='viridis', interpolation='nearest',aspect=5)
        plt.colorbar()  # adding color bar to show the scale
        plt.title("Heatmap of voltage on 4 channels")
        plt.xlabel(f"Bin Value:{bintime_ns: .3g} ns    ;    Mean SNR:{event['mean_integrated_power']: .3g}")
        y_ticks = [0, 1, 2, 3]  # Custom y-tick positions
        y_labels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4']  # Custom y-tick labels
        plt.yticks(y_ticks, y_labels)
        plt.show()

    return binned_dict

In [8]:
def plot_image(event):
    matrix = event['data']
    bintime_ns = event['bin_time']

    plt.figure(figsize=(10, 6))
    # Creating the heatmap
    plt.imshow(matrix, cmap='viridis', interpolation='nearest',aspect=5)
    plt.colorbar()  # adding color bar to show the scale
    plt.title("Heatmap of voltage on 4 channels")
    plt.xlabel(f"Bin Value:{bintime_ns: .3g} ns    ;    Mean SNR:{event['mean_SNR']: .3g}")
    y_ticks = [0, 1, 2, 3]  # Custom y-tick positions
    y_labels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4']  # Custom y-tick labels
    plt.yticks(y_ticks, y_labels)
    plt.show()

In [9]:
import os
import pickle

event_data_to_append = {}

def save_events(file_path='data/event_data.pkl',events_in=None):
    """
    Save events in a dictionary. If a dictionary already exists, it will append the events to the end of the dictionary.
    Otherwise, it will create the pickled dictionary in the file_path.

    Parameters:
    file_path (String) location of dictionary, or location where to create it.
    events_in (dict) python dictionary from which to append the events or to save.
    """
    # Check if the file exists
    if os.path.exists(file_path):
        # Load and return the dictionary
        with open(file_path, 'rb') as file:
            event_dict = pickle.load(file)

        start_i = max(event_dict.keys())

        # Function to shift keys by a given number
        def shift_keys(d, shift_amount):
            return {k + shift_amount: v for k, v in d.items()}
        
        shifted_events_in = shift_keys(events_in,start_i+1)
        new_event_dict = {**event_dict, **shifted_events_in}

        with open(file_path, 'wb') as file:
            pickle.dump(new_event_dict, file)
    else:
        # Save the event dictionary to file
        with open(file_path, 'wb') as file:
            pickle.dump(events_in, file)

In [10]:
def calculate_SNR(matrix, plot = False):
    SNR = 0
    for i,channel in enumerate(matrix):
        SNR += np.max(channel)/calculate_noise(channel, plot)

    SNR_mean = SNR/4

    return SNR_mean

In [11]:
def plot_raw(event):
    # Create subplots: 2 rows, 2 columns (for 4 channels)
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.flatten()  # Flatten the axes array for easier indexing

    event = event['data']

    # Loop through the first 4 channels and plot their data
    for i,ch in enumerate(event):  # Assuming there are at least 4 channels
        time = ch[0]  # Time values for the channel
        voltage = ch[1]  # Voltage values for the channel

        # Plot the time vs voltage for each channel in the appropriate subplot
        axes[i].plot(time, voltage)
        axes[i].set_title(f'Channel {i + 1}')
        axes[i].set_xlabel('Time (s)')
        axes[i].set_ylabel('Voltage (V)')
    
    # Adjust layout to avoid overlapping labels
    plt.tight_layout()
    plt.show()

# Example usage with event data
# event = [ [ [time1, voltage1], [time2, voltage2], ... ], ... ]

In [6]:
# Clear out eventbatch function. Might be useless. Lol.
import pickle
from datetime import datetime

def new_eventbatch(save_old=True, eventdata_path = '/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata'):
    """
    Create a new event batch. If save_old is True, it will save the current event batch to the event_data.pkl file.

    Parameters:
    save_old (bool): If True, saves the current event batch to eventbatch_old_year_month.pkl.
    """

    # Save the current event batch to eventbatch_old.pkl
    if save_old:
        with open(f'{eventdata_path}/eventbatch.pkl', 'rb') as file:
            old_eventbatch = pickle.load(file)

        current_date = datetime.now().strftime("%Y_%m_%d")
        with open(f'{eventdata_path}/eventbatch_old_{current_date}.pkl', 'wb') as file:
            pickle.dump(old_eventbatch, file)

    # Clear eventbatch
    eventbatch = {}

    with open(f'{eventdata_path}/eventbatch.pkl', 'wb') as file:
        pickle.dump(eventbatch, file)

In [8]:
def obtain_evb(eventbatch : str, eventdata_folder : str = '/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata'):
    """
    Quality of life function. Simply return event batch dictionary.

    Parameters:
    eventbatch (dict): Event dictionary to inspect.
    eventdata_folder (str): Folder where the eventbatch is stored.
    """

    if not eventbatch.endswith('.pkl'):
        eventbatch = f'{eventbatch}.pkl'
    with open(f'{eventdata_folder}/{eventbatch}', 'rb') as file:
        return pickle.load(file)

### TESTING:

In [11]:
import pickle

ev = obtain_evb('eventbatch')

ev.keys() 

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
# For image plotting!

with open('/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata/eventbatch.pkl', 'rb') as file:
    event_dict = pickle.load(file)

event = event_dict[0]

plot_image(event)

In [ ]:
dict = {0:'a',1:'b',2:'c',3:'d'}
dict_in = {0:'e',1:'f'}

start_i = max(dict.keys())

for k, v in dict_in.items():
    dict[start_i + k + 1] = v

print(dict)

In [14]:
import os
import sys
import pickle
sys.path.append('/data/i3home/ssued/RNOGCnn')
import utils

os.chdir('/data/i3home/ssued/RNOGCnn/function_testing')

# test_batch = {0:'a',1:'b',2:'c'}
# with open('testing_merge/test_batch.pkl', 'wb') as file:
#     pickle.dump(test_batch, file)

test_1 = {0:'d',1:'e'}
with open('testing_merge/test_1.pkl', 'wb') as file:
    pickle.dump(test_1, file)

test_2 = {0:'f',1:'g'}
with open('testing_merge/test_2.pkl', 'wb') as file:
    pickle.dump(test_2, file)

test_3 = {0:'h',1:'i'}
with open('testing_merge/test_3.pkl', 'wb') as file:
    pickle.dump(test_3, file)

#utils.merge_events(file_path='testing_merge/test_batch.pkl',input_path='testing_merge/test_1.pkl')

In [ ]:
import os
import time
import sys
import pickle
sys.path.append('/data/i3home/ssued/RNOGCnn')
import utils

os.chdir('/data/i3home/ssued/RNOGCnn/function_testing/testing_merge')

for file_name in os.listdir(os.getcwd()):
    if not file_name.endswith('batch.pkl'):
        utils.conjoin_events(input_path=file_name, file_path='test_batch.pkl')

with open('test_batch.pkl', 'rb') as file:
    test_batch = pickle.load(file)

print(test_batch.keys())
for value in test_batch.values():
    print(value)

In [21]:
import pickle

with open('/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata/eventbatch.pkl', 'rb') as file:
    evb = pickle.load(file)

evb.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [ ]:
import os
import time
import sys
import pickle
sys.path.append('/data/i3home/ssued/RNOGCnn')
import utils

os.chdir('/data/i3home/ssued/RNOGCnn/function_testing')

#test_batch = {0:'a',1:'b',2:'c'}
with open('testing_merge/test_batch.pkl', 'rb') as file:
    test_batch = pickle.load(file)
# #test_1 = {0:'d',1:'e'}
# with open('testing_merge/test_1.pkl', 'rb') as file:
#     test_1 = pickle.load(file)
#     print(test_1.keys())
# #test_2 = {0:'f',1:'g'}
# with open('testing_merge/test_2.pkl', 'rb') as file:
#     test_2 = pickle.load(file)
#     print(test_2.keys())



#merge_events(file_path='testing_merge/test_batch.pkl',input_path='testing_merge/test_2.pkl')

#time.sleep(5)

print(test_batch.keys())
for value in test_batch.values():
    print(value)

# Issue:

In [19]:
import pickle
import sys
sys.path.append('/data/i3home/ssued/RNOGCnn')
import utils

#utils.merge_events(input_path='/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata/eventbatch_2.pkl'
#                   ,file_path='/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata/eventbatch.pkl')

with open('/data/i3home/ssued/RNOGCnn/CNN_steps/eventdata/eventbatch.pkl','rb') as file:
    evb = pickle.load(file)
    
snr_values = [value['mean_SNR'] for value in evb.values()]
repeated_snr = set([snr for snr in snr_values if snr_values.count(snr) > 1])

if repeated_snr:
    print("Repeated SNR values found:")
    sum = 0
    for snr in repeated_snr:
        print(f"SNR: {snr}, Count: {snr_values.count(snr)}")
        sum += snr_values.count(snr)

    print(f"Total repeated SNR values: {sum}")
else:
    print("No repeated SNR values found.")

# IMPORTANT! FOR CHECKING REPEATED SNRS!

Repeated SNR values found:
SNR: 3.5846768572577563, Count: 4
SNR: 3.627613660699147, Count: 4
SNR: 3.659114176609958, Count: 4
SNR: 3.801126777444452, Count: 4
SNR: 3.671666622553848, Count: 4
SNR: 5.10557212776405, Count: 3
SNR: 9.598411610443145, Count: 3
SNR: 3.4721809052289956, Count: 3
SNR: 3.455244321595024, Count: 3
SNR: 3.9426966058277677, Count: 3
SNR: 11.94980191519963, Count: 4
SNR: 11.298217091891058, Count: 4
SNR: 7.5353890474058485, Count: 3
SNR: 3.709964447878656, Count: 4
SNR: 3.4917503287398084, Count: 4
SNR: 3.7632942333350297, Count: 3
SNR: 3.8127424035531856, Count: 3
SNR: 4.500213512598476, Count: 3
SNR: 28.33304718828407, Count: 3
SNR: 7.061293677141073, Count: 2
SNR: 3.7223495253909804, Count: 4
SNR: 3.5144131736592095, Count: 3
SNR: 3.5075680162013776, Count: 3
SNR: 67.82052844664614, Count: 2
SNR: 3.2799014465739846, Count: 4
SNR: 3.523519865109047, Count: 2
SNR: 6.647110084491207, Count: 2
SNR: 109.7476088776859, Count: 3
SNR: 3.509031515357264, Count: 3
SNR: 

In [62]:
import os
import pickle

def save_events_test(directory='/data/i3home/ssued/RNOGCnn/function_testing/data/', event_in = None):
    """
    Save a file in the directory with a numerical suffix based on existing files.
    If no files exist, it saves as 'trace_0.pkl'. Otherwise, it finds the highest numerical suffix,
    increments it, and saves the file.

    Parameters:
    directory (str): Directory to save the file.
    event_in (str): Event content to save in the file.
    """
    base_name = "trace"
    ext = ".pkl"
    max_suffix = -1

    # Ensure the directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Iterate through the files in the directory to find the highest numerical suffix
    for file_name in os.listdir(directory):
        if file_name.startswith(base_name) and file_name.endswith(ext):
            try:
                # Extract the number after the base_name and before the extension
                suffix = int(file_name[len(base_name) + 1: -len(ext)])
                max_suffix = max(max_suffix, suffix)
            except ValueError:
                # Ignore files that don't follow the naming pattern
                pass

    # Determine the new file name
    new_suffix = max_suffix + 1
    new_file_name = f"{base_name}_{new_suffix}{ext}"
    new_file_path = os.path.join(directory, new_file_name)

    # Save the text content to the new file
    with open(new_file_path, 'wb') as file:
        pickle.dump(event_in, file)

    print(f"File saved as: {new_file_path}")
